In [1]:
# 使用训练好的模型进行测试
# saver = tf.train.Saver()
# saver.restore(sess,path)
# print(sess.run(loss,feed_dict={x: batch_img,y: batch_labels,keep_prob:1.0}))
# 关键就是这几句，因为第一句需要保存变量，所以需要在代码内添加与训练时定义相同的变量
# 因为我实在不知道该怎么简化需要的代码，就把训练时定义的全部拷过来了
# 然后path就是存放训练好的模型的路径
# print哪里与训练时喂的东西要一致
import tensorflow as tf
import h5py
import numpy as np
import os

In [2]:
# 是用matlab来产生的h5文件，data字段存的是眼图数据 labels字段存的是误码率
def read_h5(path):
    f = h5py.File(path)
    img = np.transpose(f['data'])
    img = np.float32(img)
    labels = np.float32(f['labels'])
    return img,labels

In [3]:
# 跟着网上学的next_batch喂给placeholder
def next_batch(data, labels, batch_size):  
    index = [ i for i in range(0,len(labels)) ]  
    #np.random.shuffle(index)  # 打乱顺序
    batch_data = []
    batch_labels = [] 
    for i in range(0,batch_size):  
        data_app = np.reshape(data[index[i],:,:],[-1,360*460])# 这里使用np.reshape,而不使用tf.reshape因为使用了后者报错了，似乎是格式不对
        # 就是这个错  setting an array element with a sequence.
        batch_data.append(data_app[0])# 有[0]是因为data_app的结果是[1，165600],这样喂给placeholder时就是[batch_size,1,165600]
        # 而不是想要的[batch_size,165600]
        batch_labels.append(-np.log2(labels[index[i]]+1e-5))# 因为误码率是在0-0.3左右，区分度不高，所以就采用-log2(ber+1e-5)
        # 负号是为了让标签为正值,以2为底是感觉好随便取的，添加1e-5是为了避免误码率为0时的情况
    return batch_data, batch_labels 

In [4]:
batch_size = 32

test_path = 'F:/code/eyediagram/shibai/eyediagram_sur.h5'
sur_img,sur_labels = read_h5(test_path)


#n_batch好像没用到啊
# n_batch = train_img.shape[0] // batch_size  # 201/10

#收集参数的均值方差
def Variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.mean(var)
        tf.summary.scalar('mean',mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.squre(var-mean)))#标准差公式
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram', var)

#初始化权值        
def weight_variable(shape,name):
    initial = tf.truncated_normal(shape,stddev=0.1)#生成一个截断的正态分布
    return tf.Variable(initial,name)

# 初始化偏置值
def bias_variable(shape,name):
    initial = tf.constant(0.1,shape=shape)
    return tf.Variable(initial,name)

def conv2d(x,W):
    #x input tensor of shape `[batch批次大小, in_height长, in_width宽, in_channels通道数（黑白为1，彩色为3]`
    #W filter / kernel 滤波器卷积核tensor of shape [filter_height, filter_width, in_channels, out_channels]
    #`strides[0] = strides[3] = 1`. strides[1]代表x方向的步长，strides[2]代表y方向的步长
    #padding: A `string` from: `"SAME在外补0", "VALID不补0"`
    #2d表示二维卷积操作
    return tf.nn.conv2d(x,W,[1,1,1,1],padding='SAME')

def max_pooling_2x2(x):
    #ksize [1,x,y,1]窗口大小
    #ksize[0,3]默认为1，[1，2]为窗口大小
    # strides 步长
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

with tf.name_scope('input'):
    x = tf.placeholder(tf.float32,[None,165600],name='x_input')# 360*460
    y = tf.placeholder(tf.float32,[None,1],name = 'y_input')#ber
    with tf.name_scope('x_image'):
        #向量转换成图片形状
        #改变x的格式转为4D的向量[batch, in_height, in_width, in_channels]`
        #-1就是batch_size的大小
        x_image = tf.reshape(x,[-1,360,460,1],name='x_image')
        
#第一个卷积层        
with tf.name_scope('conv1'):
    with tf.name_scope('conv1_w'):
        #5*5的采样窗口，1/32表示输入/输出通道数
        conv1_w = weight_variable([5,5,1,16],name='conv1_w') # 32个卷积核
    with tf.name_scope('conv1_b'):
        #一个卷积核一个偏置
        conv1_b = bias_variable([16],name='conv1_b')
    #把x_image和权值向量进行卷积，再加上偏置值，然后应用于relu激活函数    
    with tf.name_scope('conv2d_1'):
        conv2d_1 = conv2d(x_image,conv1_w) + conv1_b
    with tf.name_scope('tanh'):
        h_conv_1 = tf.nn.tanh(conv2d_1)
    with tf.name_scope('conv1_pool'):
        conv1_pool = max_pooling_2x2(h_conv_1)
        
#第二个卷积层        
with tf.name_scope('conv2'):
    with tf.name_scope('conv2_w'):
        conv2_w = weight_variable([5,5,16,32],name='conv2_w')
    with tf.name_scope('conv2_b'):
        conv2_b = bias_variable([32],name='conv2_b')
    with tf.name_scope('conv2d_2'):
        conv2d_2 = conv2d(conv1_pool,conv2_w) + conv2_b
    with tf.name_scope('tanh'):
        h_conv_2 = tf.nn.tanh(conv2d_2)
    with tf.name_scope('conv2_pool'):
        conv2_pool = max_pooling_2x2(h_conv_2)
        
#第三个卷积层        
with tf.name_scope('conv3'):
    with tf.name_scope('conv3_w'):
        conv3_w = weight_variable([5,5,32,64],name='conv3_w')
    with tf.name_scope('conv3_b'):
        conv3_b = bias_variable([64],name='conv3_b')
    with tf.name_scope('conv2d_3'):
        conv2d_3 = conv2d(conv2_pool,conv3_w) + conv3_b
    with tf.name_scope('tanh'):
        h_conv_3 = tf.nn.tanh(conv2d_3)
    with tf.name_scope('conv3_pool'):
        conv3_pool = max_pooling_2x2(h_conv_3)
        
#第四个卷积层        
with tf.name_scope('conv4'):
    with tf.name_scope('conv4_w'):
        conv4_w = weight_variable([3,3,64,128],name='conv4_w')
    with tf.name_scope('conv4_b'):
        conv4_b = bias_variable([128],name='conv4_b')
    with tf.name_scope('conv2d_4'):
        conv2d_4 = conv2d(conv3_pool,conv4_w) + conv4_b
    with tf.name_scope('tanh'):
        h_conv_4 = tf.nn.tanh(conv2d_4)
    with tf.name_scope('conv4_pool'):
        conv4_pool = max_pooling_2x2(h_conv_4)
        
#第五个卷积层        
with tf.name_scope('conv5'):
    with tf.name_scope('conv5_w'):
        conv5_w = weight_variable([3,3,128,80],name='conv5_w')
    with tf.name_scope('conv5_b'):
        conv5_b = bias_variable([80],name='conv5_b')
    with tf.name_scope('conv2d_5'):
        conv2d_5 = conv2d(conv4_pool,conv5_w) + conv5_b
    with tf.name_scope('tanh'):
        h_conv_5 = tf.nn.tanh(conv2d_5)
    with tf.name_scope('conv5_pool'):
        conv5_pool = max_pooling_2x2(h_conv_5)
        
#第六个卷积层        
with tf.name_scope('conv6'):
    with tf.name_scope('conv6_w'):
        conv6_w = weight_variable([3,3,80,40],name='conv6_w')
    with tf.name_scope('conv6_b'):
        conv6_b = bias_variable([40],name='conv6_b')
    with tf.name_scope('conv2d_6'):
        conv2d_6 = conv2d(conv5_pool,conv6_w) + conv6_b
    with tf.name_scope('tanh'):
        h_conv_6 = tf.nn.tanh(conv2d_6)
    with tf.name_scope('conv6_pool'):
        conv6_pool = max_pooling_2x2(h_conv_6)
        
#第七个卷积层        
with tf.name_scope('conv7'):
    with tf.name_scope('conv7_w'):
        conv7_w = weight_variable([3,3,40,20],name='conv7_w')
    with tf.name_scope('conv7_b'):
        conv7_b = bias_variable([20],name='conv7_b')
    with tf.name_scope('conv2d_7'):
        conv2d_7 = conv2d(conv6_pool,conv7_w) + conv7_b
    with tf.name_scope('tanh'):
        h_conv_7 = tf.nn.tanh(conv2d_7)
    with tf.name_scope('conv7_pool'):
        conv7_pool = max_pooling_2x2(h_conv_7)

#360*460的图片第一次卷积后还是360*460，第一次池化后变为180*230
#第二次卷积后为180*230，第二次池化后变为了90*115
# 也不知道算对没有，但是都通过运行了，应该没问题，我是到奇数时添加1再除2
#进过上面操作后得到20张12*15的平面

#第一个全连接层        
with tf.name_scope('fc1'):
    with tf.name_scope('fc1_w'):
        fc1_w = weight_variable([3*4*20,60],name='fc1_w')
    with tf.name_scope('fc1_b'):
        fc1_b = bias_variable([60],name='fc1_b')
    #把池化层2的输出扁平化为1维
    with tf.name_scope('conv7_pool_flat'):
        conv7_pool_flat = tf.reshape(conv7_pool,[-1,3*4*20],name='conv7_pool_flat')
    with tf.name_scope('wx_plus_b'):
        wx_plus_b = tf.matmul(conv7_pool_flat,fc1_w) + fc1_b
    with tf.name_scope('tanh'):
        h_fc1 = tf.nn.tanh(wx_plus_b)
    with tf.name_scope('keep_prob_1'):
        keep_prob = tf.placeholder(tf.float32,name='keep_prob')
    with tf.name_scope('fc1_drop'):
        fc1_drop = tf.nn.dropout(h_fc1,keep_prob,name='fc1_drop')
        
#第二个全连接层    
with tf.name_scope('fc2'):
    with tf.name_scope('fc2_w'):
        fc2_w = weight_variable([60,1],name='fc2_w')
    with tf.name_scope('fc2_b'):
        fc2_b = bias_variable([1],name='fc2_b')
    with tf.name_scope('wx_plus_b2'):
        wx_plus_b2 = tf.matmul(fc1_drop,fc2_w) + fc2_b
    with tf.name_scope('prediction'):
        prediction = wx_plus_b2
        

with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.square(prediction - y),name='loss')
    tf.summary.scalar('loss',loss)
    
with tf.name_scope('train'):
    train = tf.train.AdamOptimizer(1e-4).minimize(loss)
    
merged = tf.summary.merge_all()
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    batch_img,batch_labels = next_batch(sur_img,sur_labels,batch_size)
    saver.restore(sess,'F:/code/eyediagram/my_eye/outcome/715/net/eyesight_net.ckpt')
    print(sess.run(loss,feed_dict={x: batch_img,y: batch_labels,keep_prob:1.0}))
    sur_prediction = sess.run(prediction,feed_dict = {x:batch_img,y:batch_labels,keep_prob:1.0})
    print(sur_prediction)

INFO:tensorflow:Restoring parameters from F:/code/eyediagram/my_eye/outcome/715/net/eyesight_net.ckpt
0.21540618
[[ 7.9948163]
 [10.243273 ]
 [ 2.4962144]
 [ 9.4180765]
 [ 4.4664   ]
 [ 2.3092818]
 [ 2.3420534]
 [ 3.9479337]
 [10.477301 ]
 [10.680367 ]
 [ 2.3195014]
 [10.905551 ]
 [10.5135565]
 [ 3.9298534]
 [ 7.6457696]
 [ 2.002858 ]
 [ 3.4149704]
 [ 9.541807 ]
 [ 7.394179 ]
 [10.3097315]
 [ 5.6459136]
 [ 2.1599026]
 [ 8.209622 ]
 [ 9.68728  ]
 [ 5.270032 ]
 [ 5.501942 ]
 [ 5.8692575]
 [ 3.10554  ]
 [ 5.279142 ]
 [ 2.4914703]
 [ 5.288733 ]
 [ 2.3520343]]


In [5]:
print(-np.log2(sur_labels)+1e-5)

[[ 7.234611 ]
 [ 9.200259 ]
 [ 2.0581825]
 [ 9.802295 ]
 [ 4.821748 ]
 [ 1.9929868]
 [ 2.5516586]
 [ 4.0214043]
 [11.086088 ]
 [10.90921  ]
 [ 2.1316392]
 [11.361723 ]
 [11.150218 ]
 [ 3.5820901]
 [ 7.175022 ]
 [ 2.0993807]
 [ 3.1718984]
 [ 9.63237  ]
 [ 6.9986258]
 [11.361723 ]
 [ 5.062756 ]
 [ 1.86692  ]
 [ 8.062756 ]
 [ 9.77676  ]
 [ 5.317329 ]
 [ 6.4050794]
 [ 6.238963 ]
 [ 3.0305703]
 [ 5.039795 ]
 [ 2.1790674]
 [ 5.7673   ]
 [ 1.8534069]]


In [6]:
print(sur_labels)

[[0.00664]
 [0.0017 ]
 [0.24012]
 [0.00112]
 [0.03536]
 [0.25122]
 [0.17056]
 [0.06158]
 [0.00046]
 [0.00052]
 [0.2282 ]
 [0.00038]
 [0.00044]
 [0.0835 ]
 [0.00692]
 [0.23336]
 [0.11096]
 [0.00126]
 [0.00782]
 [0.00038]
 [0.02992]
 [0.27416]
 [0.00374]
 [0.00114]
 [0.02508]
 [0.0118 ]
 [0.01324]
 [0.12238]
 [0.0304 ]
 [0.22082]
 [0.01836]
 [0.27674]]


In [7]:
print(2**(-sur_prediction)-1e-5)

[[0.00391031]
 [0.00081503]
 [0.17723116]
 [0.00145176]
 [0.04522552]
 [0.20175086]
 [0.19721942]
 [0.0647868 ]
 [0.00069148]
 [0.00059938]
 [0.2003267 ]
 [0.00051132]
 [0.00067408]
 [0.06560396]
 [0.00498337]
 [0.24949525]
 [0.09374436]
 [0.00133162]
 [0.00593471]
 [0.00077788]
 [0.0199615 ]
 [0.22376138]
 [0.00336798]
 [0.00120294]
 [0.02590566]
 [0.02205736]
 [0.01709714]
 [0.11617212]
 [0.02574253]
 [0.17781495]
 [0.0255719 ]
 [0.19585964]]


In [8]:
print(sur_labels-(2**(-sur_prediction)-1e-5))

[[ 2.7296892e-03]
 [ 8.8497403e-04]
 [ 6.2888831e-02]
 [-3.3176132e-04]
 [-9.8655187e-03]
 [ 4.9469128e-02]
 [-2.6659414e-02]
 [-3.2068007e-03]
 [-2.3148471e-04]
 [-7.9381163e-05]
 [ 2.7873307e-02]
 [-1.3131715e-04]
 [-2.3407562e-04]
 [ 1.7896041e-02]
 [ 1.9366271e-03]
 [-1.6135246e-02]
 [ 1.7215639e-02]
 [-7.1620918e-05]
 [ 1.8852940e-03]
 [-3.9788269e-04]
 [ 9.9585019e-03]
 [ 5.0398618e-02]
 [ 3.7201750e-04]
 [-6.2937033e-05]
 [-8.2566217e-04]
 [-1.0257360e-02]
 [-3.8571414e-03]
 [ 6.2078834e-03]
 [ 4.6574697e-03]
 [ 4.3005049e-02]
 [-7.2118957e-03]
 [ 8.0880374e-02]]


In [9]:
print(np.mean(sur_labels-(2**(-sur_prediction)-1e-5)))

0.009334373
